In [1]:
import xml.dom.minidom
import json
import re

In [10]:
last_question_id = "question_id"
map_answer = {}
with open("Eval_Scripts/data/QA_testSet_ground_truth_round_2.txt") as f:
    for i, line in enumerate(f.readlines()):
        question_id, answer_id, label = line.strip().split(",")
        if last_question_id != question_id:
            assert question_id not in map_answer.keys()
            assert label == '1'
            map_answer.update({question_id: answer_id})
                
        last_question_id = question_id

In [19]:
def convert(root, out_file):
    with open(out_file, 'w') as outfile:
        for quetion in root.getElementsByTagName('Question'):
            assert len(quetion.attributes.items())==1
            assert len(quetion.attributes.items()[0])==2
            qid = quetion.attributes.items()[0][1]
            best_aid = map_answer[qid]
            q = quetion.getElementsByTagName("QuestionText")[0].childNodes[0].data.strip()
            q = re.sub("\u00a0", " ", q)
            q_items = q.split(". ")
            if len(q_items) > 1:
                q = ". ".join(q_items[1:]).strip()

            url = ''
            answer = ''
            for a in quetion.getElementsByTagName("Answer"):
                assert len(a.attributes.items()) == 4
                aid = a.attributes.items()[0][1]
                if aid == best_aid:
                    url = a.getElementsByTagName("AnswerURL").item(0).childNodes[0].data.strip()
                    answer = a.getElementsByTagName("AnswerText").item(0).childNodes[0].data.strip()
                    break
            answer = re.sub("\u00a0", " ", answer)

            assert url and answer

            subject = ''
            a_items = answer.split(":")
            if len(a_items) > 1:
                subject = a_items[0].strip()
                answer = ":".join(a_items[1:]).strip()


            entry = {'id': qid,
                    'subject': subject,
                     'url': [url],
                     'context': "",
                     'question': q, 
                     'answer': [answer]}

            json.dump(entry, outfile)
            outfile.write('\n')

In [12]:
dom = xml.dom.minidom.parse('MEDIQA_Task3_QA/MEDIQA2019-Task3-QA-TestSet-wLabels.xml')
root = dom.documentElement
convert(root, 'test_data.jsonl')

In [ ]:
dom = xml.dom.minidom.parse('MEDIQA_Task3_QA/MEDIQA2019-Task3-QA-ValidationSet.xml')
root = dom.documentElement
convert(root, 'val_data.jsonl')